In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%autosave 10

gpu_id='gpu2'
import os
os.environ["THEANO_FLAGS"] = "device="+gpu_id +",lib.cnmem=1"

import numpy as np
from keras.layers.advanced_activations import SReLU
from keras import callbacks as ckbs
import random
import time
import gzip
from keras.models import model_from_json
import pickle as pkl
import keras
from keras.models import Sequential
from keras.layers.core import *
from keras.layers.wrappers import *
from keras.optimizers import *
from keras.utils import np_utils
from keras.layers.embeddings import Embedding
from keras.layers import LSTM
from docutils.languages.af import labels
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

Autosaving every 10 seconds


Using Theano backend.
Using gpu device 2: Tesla K80 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 4007)


In [2]:
f = gzip.open('../data/processed/pkl_short/embeddings.pkl.gz', 'rb')
embeddings = pkl.load(f)
f.close()

label2Idx = embeddings['label2Idx']

#Inverse label mapping
idx2Label = {v: k for k, v in label2Idx.items()}

f = gzip.open('../data/processed/pkl/data.pkl.gz', 'rb')
test_data = pkl.load(f)
dev_data = pkl.load(f)
wiki_data = pkl.load(f)
f.close()

n_out = len(label2Idx)

In [6]:
def get_data_for_testing(matrix, target):
   
    train_samples = int(matrix.shape[0]*1)
    X_train = matrix[:train_samples]
    X_test = matrix[train_samples:]
        
    y_train = target[:train_samples]
    y_test = target[train_samples:]
        
    
    
    return  [X_train[:,:,i] for i in range(X_train.shape[2])], \
            y_train

In [4]:
def load_neural_network(file_from):
    (nn_arch, nn_weights_path) = pkl.load(open(file_from, 'rb'))
    nn = model_from_json(nn_arch)
    nn.set_weights(nn_weights_path)
    return nn

In [8]:
def get_data(data):
    print( "%d test size" % len(data))
    data = np.array(data)
    print(data.shape)
    X = [x[0] for x in data]
    y = [x[1][0] for x in data]
    return np.array(X), np.array(y)

In [10]:
from sklearn.metrics import classification_report
X_test,y_test = get_data(test_data)
X_test,y_test = get_data_for_testing(X_test,y_test)

59381 test size
(59381, 2)


This function load model and make test (classification report)

In [26]:
def process(model_name):
    model = load_neural_network('../models/' + model_name)
    pred = model.predict_classes(X_test, batch_size=128, verbose=1)
    target_names = [idx2Label[1], idx2Label[0], idx2Label[2], idx2Label[3]]
    print(classification_report(y_test, pred, target_names=target_names))
    return model

In [21]:
process('new1')

59381/59381 [==============================] - 24s    
             precision    recall  f1-score   support

          0       0.45      0.73      0.56      1591
          1       0.96      0.98      0.97     51936
          2       0.87      0.69      0.77      2106
          3       0.64      0.34      0.45      3748

avg / total       0.92      0.92      0.92     59381



In [23]:
process('new4')

59381/59381 [==============================] - 20s    
             precision    recall  f1-score   support

          0       0.61      0.56      0.58      1591
          1       0.95      0.98      0.96     51936
          2       0.74      0.82      0.78      2106
          3       0.64      0.36      0.46      3748

avg / total       0.91      0.92      0.92     59381



In [24]:
process('new2')

59381/59381 [==============================] - 20s    
             precision    recall  f1-score   support

          0       0.56      0.52      0.54      1591
          1       0.95      0.98      0.97     51936
          2       0.80      0.63      0.70      2106
          3       0.53      0.38      0.44      3748

avg / total       0.91      0.92      0.91     59381



In [28]:
m = process('new5')

59381/59381 [==============================] - 20s    
             precision    recall  f1-score   support

        ORG       0.61      0.68      0.64      1591
          O       0.95      0.98      0.97     51936
        PER       0.81      0.83      0.82      2106
        LOC       0.68      0.31      0.43      3748

avg / total       0.92      0.93      0.92     59381



In [29]:
m.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_101 (Embedding)        (None, 33, 300)       1903200                                      
____________________________________________________________________________________________________
embedding_102 (Embedding)        (None, 33, 300)       915300                                       
____________________________________________________________________________________________________
embedding_103 (Embedding)        (None, 33, 300)       265200                                       
____________________________________________________________________________________________________
embedding_104 (Embedding)        (None, 33, 10)        1020                                         
___________________________________________________________________________________________

In [30]:
m = process('new6')

59381/59381 [==============================] - 21s    
             precision    recall  f1-score   support

        ORG       0.41      0.77      0.53      1591
          O       0.97      0.96      0.97     51936
        PER       0.79      0.79      0.79      2106
        LOC       0.56      0.38      0.46      3748

avg / total       0.92      0.92      0.92     59381

